In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("transform") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

* Without .getOrCreate()  
If you don't use .getOrCreate(), a new Spark session will be created each time you execute this code, even if a Spark session with the same configuration already exists.

In [5]:
spark.sql('SHOW TABLES IN f1_processed').show()

+------------+------------+-----------+
|   namespace|   tableName|isTemporary|
+------------+------------+-----------+
|f1_processed|    circuits|      false|
|f1_processed|constructors|      false|
|f1_processed|     drivers|      false|
|f1_processed|   lap_times|      false|
|f1_processed|  qualifying|      false|
|f1_processed|       races|      false|
|f1_processed|     results|      false|
+------------+------------+-----------+



for points for positions had changed over years. so to automate that the calculated column is made

In [6]:
spark.sql('CREATE TABLE f1_processed.calculated_race_results USING parquet AS SELECT races.race_year, constructors.name AS team_name, drivers.name AS driver_name, results.position, results.points,11 - results.position AS calculated_points FROM f1_processed.results JOIN f1_processed.drivers ON (results.driver_id = drivers.driver_id)   JOIN f1_processed.constructors ON (results.constructor_id = constructors.constructor_id)   JOIN f1_processed.races ON (results.race_id = races.race_id)  WHERE results.position <= 10')

DataFrame[]

```
CREATE TABLE f1_processed.calculated_race_results
USING parquet
AS
SELECT races.race_year,
       constructors.name AS team_name,
       drivers.name AS driver_name,
       results.position,
       results.points,
       11 - results.position AS calculated_points
  FROM results 
  JOIN f1_processed.drivers ON (results.driver_id = drivers.driver_id)
  JOIN f1_processed.constructors ON (results.constructor_id = constructors.constructor_id)
  JOIN f1_processed.races ON (results.race_id = races.race_id)
 WHERE results.position <= 10
 ```

### Dominant team  
The avg score of tam is calculated to find the dominant team

In [8]:
spark.sql('SELECT team_name, COUNT(1) AS total_races, SUM(calculated_points) AS total_points, AVG(calculated_points) AS avg_points FROM f1_processed.calculated_race_results WHERE race_year BETWEEN 2011 AND 2020 GROUP BY team_name HAVING COUNT(1) >= 100 ORDER BY avg_points DESC').show()

+-----------+-----------+------------+------------------+
|  team_name|total_races|total_points|        avg_points|
+-----------+-----------+------------+------------------+
|   Mercedes|        331|        2580| 7.794561933534744|
|   Red Bull|        313|        2201| 7.031948881789138|
|    Ferrari|        314|        2095| 6.671974522292993|
|    McLaren|        204|         984| 4.823529411764706|
|   Williams|        125|         531|             4.248|
|Force India|        189|         671|3.5502645502645502|
| Toro Rosso|        119|         316|2.6554621848739495|
+-----------+-----------+------------+------------------+



```
SELECT team_name,
       COUNT(1) AS total_races,
       SUM(calculated_points) AS total_points,
       AVG(calculated_points) AS avg_points
  FROM f1_processed.calculated_race_results
 WHERE race_year BETWEEN 2011 AND 2020
GROUP BY team_name
HAVING COUNT(1) >= 100
ORDER BY avg_points DESC
```

### Dominant Driver

In [9]:
spark.sql('SELECT driver_name, COUNT(1) AS total_races, SUM(calculated_points) AS total_points, AVG(calculated_points) AS avg_points FROM f1_processed.calculated_race_results WHERE race_year BETWEEN 2001 AND 2010 GROUP BY driver_name HAVING COUNT(1) >= 50 ORDER BY avg_points DESC').show()

+--------------------+-----------+------------+------------------+
|         driver_name|total_races|total_points|        avg_points|
+--------------------+-----------+------------+------------------+
|  Michael Schumacher|        104|         832|               8.0|
|      Lewis Hamilton|         55|         425|7.7272727272727275|
|  Juan Pablo Montoya|         58|         427| 7.362068965517241|
|     Fernando Alonso|        114|         827| 7.254385964912281|
|      Kimi Räikkönen|        106|         758| 7.150943396226415|
|  Rubens Barrichello|        113|         696|  6.15929203539823|
|       Jenson Button|        102|         601| 5.892156862745098|
|        Felipe Massa|         93|         533| 5.731182795698925|
|     Ralf Schumacher|         70|         399|               5.7|
|       Robert Kubica|         51|         283| 5.549019607843137|
|     David Coulthard|         79|         424| 5.367088607594937|
|        Jarno Trulli|         83|         409| 4.927710843373

```
SELECT driver_name,
       COUNT(1) AS total_races,
       SUM(calculated_points) AS total_points,
       AVG(calculated_points) AS avg_points
  FROM f1_processed.calculated_race_results
 WHERE race_year BETWEEN 2001 AND 2010
GROUP BY driver_name
HAVING COUNT(1) >= 50
ORDER BY avg_points DESC

```